In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from tqdm import tqdm
import time

train = pd.read_csv("CSV_train.csv",low_memory=False,delimiter=';')
test=pd.read_csv("CSV_test.csv",low_memory=False,delimiter=',')
hidden=pd.read_csv("CSV_hidden_test.csv",low_memory=False,delimiter=',')

In [2]:
#Storing dataset wells and depths
train_well = train.WELL.values; train_depth = train.DEPTH_MD.values
test_well = test.WELL.values; test_depth = test.DEPTH_MD.values
hidden_well = hidden.WELL.values; hidden_depth = hidden.DEPTH_MD.values

In [3]:
# storing length of datasets 
train_len = train.shape[0] 
test_len = test.shape[0]
All_data = pd.concat((train,test,hidden)).reset_index(drop=True) 

lithology_keys = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}
All_data['Lithology'] = All_data['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_keys)
All_data

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0,Shale
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0,Shale
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0,Shale
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0,Shale
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0,Shale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,136.911575,NaN,0.502458,NaN,2.311106,24.306124,NaN,65000,2.0,Shale
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,137.583923,NaN,0.374753,NaN,1.853418,22.201078,NaN,65000,2.0,Shale
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,138.310898,NaN,0.211487,NaN,1.325961,20.096741,NaN,65000,2.0,Shale
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,137.592819,NaN,0.147950,NaN,1.260347,17.992323,NaN,65000,2.0,Shale


In [4]:
#dropping columns with high missing values
drop_cols = ['SGR', 'ROPA', 'RXO', 'MUDWEIGHT','DCAL','RMIC','FORCE_2020_LITHOFACIES_CONFIDENCE']
All_data_drop = All_data.drop(drop_cols, axis=1)
# encoding categorical variables
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP'].astype('category')
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP_encoded'].cat.codes

All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION'].astype('category')
All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION_encoded'].cat.codes

All_data_drop['WELL_encoded'] = All_data_drop['WELL'].astype('category')
All_data_drop['WELL_encoded'] = All_data_drop['WELL_encoded'].cat.codes

All_data_drop['Lithology_encoded'] = All_data_drop['Lithology'].astype('category')
All_data_drop['Lithology_encoded'] = All_data_drop['Lithology_encoded'].cat.codes

In [5]:
All_data_drop

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,BS,ROP,DTS,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY,Lithology,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,34.636410,NaN,-0.574928,65000,Shale,6,-1,1,10
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,34.636410,NaN,-0.570188,65000,Shale,6,-1,1,10
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,34.779556,NaN,-0.574245,65000,Shale,6,-1,1,10
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,39.965164,NaN,-0.586315,65000,Shale,6,-1,1,10
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,57.483765,NaN,-0.597914,65000,Shale,6,-1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,8.5,15.195305,136.911575,0.502458,65000,Shale,0,16,113,10
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,8.5,15.770223,137.583923,0.374753,65000,Shale,0,16,113,10
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,8.5,16.418465,138.310898,0.211487,65000,Shale,0,16,113,10
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,8.5,17.037945,137.592819,0.147950,65000,Shale,0,16,113,10


In [6]:
#dropping categorial features replaces beforehan by encoded features
drop2 = All_data_drop.drop(['GROUP', 'FORMATION','WELL','FORCE_2020_LITHOFACIES_LITHOLOGY','Lithology'], axis=1)

# splitting dataset into training, test, and hidden sets
train_prep = drop2[:train_len].copy()
test_prep = drop2[train_len:(train_len+test_len)].copy()
hidden_prep = drop2[(train_len+test_len):].copy()

In [7]:
drop2

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,NaN,34.636410,NaN,-0.574928,6,-1,1,10
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,NaN,34.636410,NaN,-0.570188,6,-1,1,10
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,NaN,34.779556,NaN,-0.574245,6,-1,1,10
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,NaN,39.965164,NaN,-0.586315,6,-1,1,10
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,NaN,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,NaN,57.483765,NaN,-0.597914,6,-1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,NaN,2.820439,3.158570,NaN,90.720284,...,75.260658,NaN,8.5,15.195305,136.911575,0.502458,0,16,113,10
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,NaN,3.020778,3.332977,NaN,87.062027,...,74.868301,NaN,8.5,15.770223,137.583923,0.374753,0,16,113,10
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,NaN,2.795711,3.044179,NaN,86.115921,...,74.848122,NaN,8.5,16.418465,138.310898,0.211487,0,16,113,10
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,NaN,NaN,2.658694,2.847681,NaN,89.497131,...,74.964027,NaN,8.5,17.037945,137.592819,0.147950,0,16,113,10


In [8]:
train_prep1= train_prep.copy()
test_prep1= test_prep.copy()
hidden_prep1= hidden_prep.copy()

In [9]:
#Inputing missing values by introducing median 
from sklearn.impute import SimpleImputer
miss = SimpleImputer(missing_values=np.nan, strategy='median')
miss.fit(drop2)
All_imp = miss.fit_transform(drop2)
All_imp=pd.DataFrame(All_imp, columns=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO',
         'GROUP_encoded',
       'FORMATION_encoded', 'WELL_encoded','Lithology_encoded'])
All_imp

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,1.398049,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,12.250001,34.636410,189.362198,-0.574928,6.0,-1.0,1.0,10.0
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,1.398049,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,12.250001,34.636410,189.362198,-0.570188,6.0,-1.0,1.0,10.0
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,1.398049,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,12.250001,34.779556,189.362198,-0.574245,6.0,-1.0,1.0,10.0
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,1.398049,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,12.250001,39.965164,189.362198,-0.586315,6.0,-1.0,1.0,10.0
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,1.398049,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,12.250001,57.483765,189.362198,-0.597914,6.0,-1.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,1.398049,2.820439,3.158570,2.331407,90.720284,...,75.260658,54.270451,8.500000,15.195305,136.911575,0.502458,0.0,16.0,113.0,10.0
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,1.398049,3.020778,3.332977,2.331407,87.062027,...,74.868301,54.270451,8.500000,15.770223,137.583923,0.374753,0.0,16.0,113.0,10.0
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,1.398049,2.795711,3.044179,2.331407,86.115921,...,74.848122,54.270451,8.500000,16.418465,138.310898,0.211487,0.0,16.0,113.0,10.0
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,12.515673,1.398049,2.658694,2.847681,2.331407,89.497131,...,74.964027,54.270451,8.500000,17.037945,137.592819,0.147950,0.0,16.0,113.0,10.0


In [10]:
# print(All_imp['GR'].quantile(0.50)) 
# print(All_imp['GR'].quantile(0.95)) 
# All_imp['GR'] = np.where(All_imp['GR'] > 150, 67, All_imp['GR'])
# All_imp['GR'].describe()

In [11]:
All_imp.describe()

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
count,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,...,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06,1.429694e+06
mean,2.216632e+03,4.846857e+05,6.680881e+06,-2.166097e+03,1.306219e+01,5.929846e+00,4.633407e+00,1.105773e+01,2.297625e+00,7.047302e+01,...,1.122697e+02,5.615993e+01,1.198030e+01,6.489179e+01,1.935923e+02,1.135653e-02,6.681300e+00,3.683317e+01,5.681314e+01,9.139322e+00
std,9.929555e+02,3.509749e+04,1.311172e+05,9.554058e+02,3.575119e+00,7.092686e+01,5.001560e+01,1.067479e+02,2.336318e-01,3.408658e+01,...,2.886977e+01,6.174737e+01,2.396487e+00,9.435014e+02,3.622618e+01,6.215910e+00,3.148458e+00,2.368314e+01,3.279013e+01,1.501331e+00
min,1.360860e+02,4.232375e+05,6.406641e+06,-5.395563e+03,2.344000e+00,1.000000e-04,-8.418695e-03,3.170056e-02,7.209712e-01,1.092843e-01,...,7.415132e+00,-9.990000e+02,6.000000e+00,-1.179770e-01,6.916318e+01,-7.429339e+03,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,1.457365e+03,4.544695e+05,6.590310e+06,-2.833309e+03,9.973806e+00,1.398049e+00,9.265758e-01,9.104230e-01,2.131772e+00,4.737605e+01,...,8.842327e+01,4.287162e+01,1.225000e+01,1.929947e+01,1.893622e+02,-5.890843e-03,5.000000e+00,1.800000e+01,3.200000e+01,9.000000e+00
50%,2.121598e+03,4.786488e+05,6.737678e+06,-2.088677e+03,1.251567e+01,1.398049e+00,1.455281e+00,1.453520e+00,2.331407e+00,6.786042e+01,...,1.083101e+02,5.427045e+01,1.225000e+01,1.929947e+01,1.893622e+02,3.207336e-03,6.000000e+00,3.700000e+01,5.500000e+01,1.000000e+01
75%,2.895131e+03,5.201532e+05,6.784878e+06,-1.435200e+03,1.521438e+01,1.398049e+00,2.593216e+00,2.593180e+00,2.471782e+00,8.924216e+01,...,1.387327e+02,6.511991e+01,1.225000e+01,1.929947e+01,1.893622e+02,1.877124e-02,9.000000e+00,5.800000e+01,8.300000e+01,1.000000e+01
max,5.436632e+03,5.726328e+05,6.856661e+06,-1.110860e+02,2.827900e+01,2.193905e+03,1.988616e+03,1.999887e+03,3.457820e+00,1.141292e+03,...,3.204789e+02,5.265473e+02,2.600000e+01,4.701512e+04,6.765781e+02,2.836938e+00,1.300000e+01,6.900000e+01,1.170000e+02,1.100000e+01


In [12]:
All_imp.clip(lower=All_imp.quantile(0.05), upper=All_imp.quantile(0.98), axis=1)

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,729.48095,437780.34375,6470972.5,-518.136710,19.480835,1.398049,1.611410,1.798681,1.899048,80.200851,...,161.131180,24.612379,12.250001,34.636410,189.362198,-0.031164,6.0,-1.0,3.0,10.0
1,729.48095,437780.34375,6470972.5,-518.136710,19.468800,1.398049,1.618070,1.795641,1.899048,79.262886,...,160.603470,23.895531,12.250001,34.636410,189.362198,-0.031164,6.0,-1.0,3.0,10.0
2,729.48095,437780.34375,6470972.5,-518.136710,19.468800,1.398049,1.626459,1.800733,1.899048,74.821999,...,160.173615,23.916357,12.250001,34.779556,189.362198,-0.031164,6.0,-1.0,3.0,10.0
3,729.48095,437780.34375,6470972.5,-518.136710,19.459282,1.398049,1.621594,1.801517,1.899048,72.878922,...,160.149429,23.793688,12.250001,39.965164,189.362198,-0.031164,6.0,-1.0,3.0,10.0
4,729.48095,437780.34375,6470972.5,-518.136710,19.453100,1.398049,1.602679,1.795299,1.899048,71.729141,...,160.128342,24.104078,12.250001,57.483765,189.362198,-0.031164,6.0,-1.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.29880,536096.06250,6793022.0,-2943.444580,8.409431,1.398049,2.820439,3.158570,2.331407,90.720284,...,75.260658,54.270451,8.500000,15.195305,154.237827,0.121060,2.0,16.0,113.0,10.0
1429690,2973.45080,536096.06250,6793022.0,-2943.595947,8.409431,1.398049,3.020778,3.332977,2.331407,87.062027,...,74.868301,54.270451,8.500000,15.770223,154.237827,0.121060,2.0,16.0,113.0,10.0
1429691,2973.60280,536096.06250,6793022.0,-2943.747559,8.409431,1.398049,2.795711,3.044179,2.331407,86.115921,...,74.848122,54.270451,8.500000,16.418465,154.237827,0.121060,2.0,16.0,113.0,10.0
1429692,2973.75480,536096.06250,6793022.0,-2943.899170,12.515673,1.398049,2.658694,2.847681,2.331407,89.497131,...,74.964027,54.270451,8.500000,17.037945,154.237827,0.121060,2.0,16.0,113.0,10.0


In [13]:
train_imp = All_imp[:train_len].copy()
test_imp = All_imp[train_len:(train_len+test_len)].copy()
hidden_imp = All_imp[(train_len+test_len):].copy()

In [14]:
print(train_imp.shape)
print(test_imp.shape)
print(hidden_imp.shape)

(1170511, 22)
(136786, 22)
(122397, 22)


In [15]:
train_imp

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,1.398049,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,12.250001,34.636410,189.362198,-0.574928,6.0,-1.0,1.0,10.0
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,1.398049,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,12.250001,34.636410,189.362198,-0.570188,6.0,-1.0,1.0,10.0
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,1.398049,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,12.250001,34.779556,189.362198,-0.574245,6.0,-1.0,1.0,10.0
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,1.398049,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,12.250001,39.965164,189.362198,-0.586315,6.0,-1.0,1.0,10.0
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,1.398049,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,12.250001,57.483765,189.362198,-0.597914,6.0,-1.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170506,3169.3124,478648.81250,6737678.5,-2088.677002,8.423170,1.398049,1.455281,1.453520,2.527984,77.654900,...,108.310127,54.270451,8.500000,27.674368,189.362198,-0.001763,11.0,7.0,117.0,8.0
1170507,3169.4644,478648.81250,6737678.5,-2088.677002,8.379244,1.398049,1.455281,1.453520,2.537613,75.363937,...,108.310127,54.270451,8.500000,28.024338,189.362198,-0.007600,11.0,7.0,117.0,9.0
1170508,3169.6164,478648.81250,6737678.5,-2088.677002,8.350248,1.398049,1.455281,1.453520,2.491860,66.452843,...,108.310127,54.270451,8.500000,28.091282,189.362198,-0.018297,11.0,7.0,117.0,9.0
1170509,3169.7684,478648.81250,6737678.5,-2088.677002,8.313779,1.398049,1.455281,1.453520,2.447539,55.784817,...,108.310127,54.270451,8.500000,28.019775,189.362198,-0.011438,11.0,7.0,117.0,9.0


In [16]:
train_imp.describe()

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
count,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,...,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06
mean,2.184087e+03,4.855668e+05,6.681795e+06,-2.138068e+03,1.313538e+01,6.406896e+00,4.869328e+00,1.060410e+01,2.291382e+00,7.091370e+01,...,1.130094e+02,5.852595e+01,1.206421e+01,7.327165e+01,1.916435e+02,1.079367e-02,6.619940e+00,3.644059e+01,5.752154e+01,9.169452e+00
std,9.971821e+02,3.440346e+04,1.276749e+05,9.664750e+02,3.657786e+00,7.401863e+01,5.375806e+01,1.134141e+02,2.357325e-01,3.423149e+01,...,2.896434e+01,6.584702e+01,2.548096e+00,1.042454e+03,2.798447e+01,6.869625e+00,3.136000e+00,2.394241e+01,3.196306e+01,1.455014e+00
min,1.360860e+02,4.268988e+05,6.406641e+06,-5.395563e+03,2.344000e+00,1.000000e-04,-8.418695e-03,3.170056e-02,7.209712e-01,1.092843e-01,...,7.415132e+00,-9.990000e+02,6.000000e+00,-1.179770e-01,6.916318e+01,-7.429339e+03,-1.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00
25%,1.418597e+03,4.548019e+05,6.593126e+06,-2.804552e+03,9.882808e+00,1.296196e+00,9.290146e-01,9.148633e-01,2.123109e+00,4.762722e+01,...,8.907137e+01,4.205721e+01,1.225000e+01,1.929947e+01,1.893622e+02,-6.882701e-03,5.000000e+00,1.800000e+01,3.400000e+01,9.000000e+00
50%,2.076605e+03,4.777699e+05,6.737314e+06,-2.055283e+03,1.251567e+01,1.398049e+00,1.455281e+00,1.449790e+00,2.331407e+00,6.836763e+01,...,1.083101e+02,5.427045e+01,1.225000e+01,1.929947e+01,1.893622e+02,3.207336e-03,6.000000e+00,3.700000e+01,5.600000e+01,1.000000e+01
75%,2.864393e+03,5.201314e+05,6.784877e+06,-1.397964e+03,1.574931e+01,1.510922e+00,2.587100e+00,2.537876e+00,2.462978e+00,8.903551e+01,...,1.393171e+02,7.039606e+01,1.225000e+01,1.929947e+01,1.893622e+02,1.555134e-02,9.000000e+00,5.800000e+01,8.500000e+01,1.000000e+01
max,5.436632e+03,5.726328e+05,6.856661e+06,-1.110860e+02,2.827900e+01,2.193905e+03,1.988616e+03,1.999887e+03,3.457820e+00,1.076964e+03,...,3.204789e+02,5.265473e+02,2.600000e+01,4.701512e+04,6.765781e+02,2.836938e+00,1.300000e+01,6.900000e+01,1.170000e+02,1.100000e+01


In [17]:


# calculating p-impedance
train_imp['PI'] = train_imp.RHOB * (1e6/train_imp.DTC)
test_imp['PI'] = test_imp.RHOB * (1e6/test_imp.DTC)
hidden_imp['PI'] = hidden_imp.RHOB * (1e6/hidden_imp.DTC)

# calculating s-impedance
train_imp['SI'] = train_imp.RHOB * (1e6/train_imp.DTS) 
test_imp['SI'] = test_imp.RHOB * (1e6/test_imp.DTS) 
hidden_imp['SI'] = hidden_imp.RHOB * (1e6/hidden_imp.DTS) 

#calculating Shear modulus (G)
train_imp['G'] = ((1e6/train_imp.DTS)**2) * train_imp.RHOB
test_imp['G'] = ((1e6/test_imp.DTS)**2) * test_imp.RHOB
hidden_imp['G'] = ((1e6/hidden_imp.DTS)**2) * hidden_imp.RHOB

#calculating Bulk modulus (K)
train_imp['K'] = (((1e6/train_imp.DTC)**2) * train_imp.RHOB) - (4 * train_imp.G/3)
test_imp['K'] = (((1e6/test_imp.DTC)**2) * test_imp.RHOB) - (4 * test_imp.G/3)
hidden_imp['K'] = (((1e6/hidden_imp.DTC)**2) * hidden_imp.RHOB) - (4 * hidden_imp.G/3)

# calculate the shale volume
train_imp["VSHALE"] = (train_imp.GR - np.min(train_imp.GR)) / (np.max(train_imp.GR) - np.min(train_imp.GR))
test_imp["VSHALE"] = (test_imp.GR - np.min(test_imp.GR)) / (np.max(test_imp.GR) - np.min(test_imp.GR))
hidden_imp["VSHALE"] = (hidden_imp.GR - np.min(hidden_imp.GR)) / (np.max(hidden_imp.GR) - np.min(hidden_imp.GR))
#train_imp1.head()

# calculate the total porosity
train_imp['PHIT'] = np.sqrt(((((train_imp.NPHI)*(train_imp.NPHI)+(train_imp.RHOB)*(train_imp.RHOB))))/2)
test_imp['PHIT'] = np.sqrt(((((test_imp.NPHI)*(test_imp.NPHI)+(test_imp.RHOB)*(test_imp.RHOB))))/2)
hidden_imp['PHIT'] = np.sqrt(((((hidden_imp.NPHI)*(hidden_imp.NPHI)+(hidden_imp.RHOB)*(hidden_imp.RHOB))))/2)
#train_imp1.tail()

# calculate effective porosity
train_imp['PHIE'] = train_imp.PHIT*(1-train_imp.VSHALE)
train_imp = train_imp[train_imp['PHIE'] !=0]
train_imp['PHIE'] = train_imp['PHIE'].abs() 

test_imp['PHIE'] = test_imp.PHIT*(1-test_imp.VSHALE)
test_imp = test_imp[test_imp['PHIE'] !=0]
test_imp['PHIE'] = test_imp['PHIE'].abs() 

hidden_imp['PHIE'] = hidden_imp.PHIT*(1-hidden_imp.VSHALE)
hidden_imp = hidden_imp[hidden_imp['PHIE'] !=0]
hidden_imp['PHIE'] = hidden_imp['PHIE'].abs() 
#train_imp1.tail()

# # display the log for preview
# plt.figure(figsize=(5,10))
# plt.subplot(122)
# plt.title('PHIE')
# plt.plot('PHIE', 'SI','PI','G','K','VSHALE','DEPTH_MD', data=train_imp)
# plt.gca().invert_yaxis()

C:\Users\Ayori\AppData\Local\Temp/ipykernel_6172/3060712503.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_imp['PHIE'] = train_imp['PHIE'].abs()


In [18]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
x_header=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO','K','G','VSHALE','PHIT','PHIE','SI','PI',
       'GROUP_encoded', 'FORMATION_encoded', 'WELL_encoded']
y_header=['Lithology_encoded']
x_train = train_imp[x_header]
y_train = train_imp[y_header]
x_test = test_imp[x_header]
y_test = test_imp[y_header]
x_hidden = hidden_imp[x_header]
y_hidden = hidden_imp[y_header]

##Min-Max scaler 
scaler = MinMaxScaler()
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()
x_hidden_scaled = x_hidden.copy()

x_train_scaled.iloc[:,:25] = scaler.fit_transform(x_train_scaled.iloc[:,:25])
x_test_scaled.iloc[:,:25] = scaler.transform(x_test_scaled.iloc[:,:25])
x_hidden_scaled.iloc[:,:25] = scaler.transform(x_hidden_scaled.iloc[:,:25])

In [34]:
x_train_scaled=x_train_scaled.drop(labels=0,axis=0)


In [35]:
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]
    try:
        print(X[0])
    except IndexError:
        print('array is empty')

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug

# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad

#Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        try:
            print(X[0])
        except IndexError:
            print('array is empty')
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [36]:
x_train_arg, padded_rows = augment_features(x_train_scaled.values, train_well, train_depth)
x_test_arg, padded_rows = augment_features(x_test_scaled.values, test_well, test_depth)
x_hidden_arg, padded_rows = augment_features(x_hidden_scaled.values, hidden_well, hidden_depth)

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.371

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[ 1.12910104e-01  3.31977338e-01  2.48373406e-01  8.86558300e-01
  4.02814237e-01  6.37196795e-04  7.05627673e-04  7.16246923e-04
  5.88427157e-01  1.19863906e-01  3.47271949e-01  1.11379682e-02
  4.71340323e-01  6.90421365e-01  1.93749982e-01  5.58361608e-04
  1.97886175e-01  9.99618721e-01  8.68659029e-03  1.29832418e-01
  1.19863906e-01  5.62575526e-01  5.97472349e-01  3.11316373e-01
  3.63158882e-02  6.00000000e+00 -1.00000000e+00  1.20000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.371

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[1.60073642e-01 2.67268034e-01 3.49972223e-01 8.39441788e-01
 3.71835981e-01 6.04543926e-04 6.81110705e-04 6.63305051e-04
 3.43337086e-01 6.71526070e-02 3.47271949e-01 2.07957914e-01
 3.22282587e-01 6.90421365e-01 3.12500018e-01 8.17466370e-04
 1.97886175e-01 9.99618721e-01 1.06341593e-02 8.97247008e-02
 6.71526070e-02 3.03879494e-01 4.48094884e-01 1.92637721e-01
 3.74383579e-02 5.00000000e+00 5.20000000e+01 2.80000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[3.37452406e-01 4.31782269e-01 4.35617306e-01 6.61999030e-01
 6.17428361e-01 6.37196795e-04 3.38776021e-04 4.62114783e-04
 5.91411343e-01 6.97422413e-02 3.47271949e-01 1.11379682e-02
 3.77977743e-01 6.55791942e-01 5.74999945e-01 3.04770585e-01
 1.97886175e-01 9.99619504e-01 1.01055775e-02 1.30320764e-01
 6.97422413e-02 5.65737461e-01 6.33735980e-01 3.12761389e-01
 5.01778582e-02 5.00000000e+00 2.10000000e+01 3.90000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[ 2.28543248e-02  4.84023007e-01  7.14792676e-01  9.77076199e-01
  5.92196495e-01  9.72550284e-04  8.42851467e-04  1.09550210e-03
  5.88427157e-01  5.99067023e-02  3.47271949e-01  1.11379682e-02
  5.06771961e-01  6.85032179e-01  3.12500018e-01  4.54953455e-03
  1.97886175e-01  9.99618721e-01  8.33496864e-03  1.29832418e-01
  5.99067023e-02  5.62575526e-01  6.38152373e-01  3.11316373e-01
  3.24051190e-02  6.00000000e+00 -1.00000000e+00  5.00000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.371

...

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[ 7.16552569e-02  7.94495621e-02  9.05623083e-01  9.28330883e-01
  3.92198685e-01  3.79287538e-04  7.36036154e-04  4.07267481e-04
  5.88427157e-01  1.72898961e-02  3.47271949e-01  1.11379682e-02
  3.22282587e-01  6.90421365e-01  3.12500018e-01  4.13003245e-04
  1.97886175e-01  9.99462985e-01  1.15384757e-02  1.29832418e-01
  1.72898961e-02  5.62575526e-01  6.67067209e-01  3.11316373e-01
  6.15413956e-02  6.00000000e+00 -1.00000000e+00  7.10000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.371

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[3.74286723e-01 3.56188191e-01 9.75858851e-01 6.24632512e-01
 3.74033496e-01 6.37196795e-04 3.99584411e-04 4.00864585e-04
 5.88427157e-01 8.54250364e-02 3.47271949e-01 1.11379682e-02
 4.05793140e-01 6.90421365e-01 5.74999945e-01 5.64050439e-04
 1.97886175e-01 9.99618721e-01 9.58120650e-03 1.29832418e-01
 8.54250364e-02 5.62575526e-01 6.20838583e-01 3.11316373e-01
 4.52516984e-02 9.00000000e+00 3.40000000e+01 8.50000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[2.09297231e-01 7.10053443e-01 8.18966935e-01 7.89794314e-01
 3.94928175e-01 6.37196795e-04 4.40158346e-04 4.20585791e-04
 5.88427157e-01 6.40220199e-02 6.62927550e-01 1.11379682e-02
 3.22282587e-01 6.90421365e-01 5.74999945e-01 1.38820891e-03
 1.97886175e-01 9.99618721e-01 1.15384757e-02 1.29832418e-01
 6.40220199e-02 5.88644186e-01 6.54076967e-01 3.11316373e-01
 6.15413956e-02 5.00000000e+00 6.80000000e+01 9.60000000e+01]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8

[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8.82034150e-04
  4.27068664e-01  7.62642844e-02  3.47271949e-01  5.03440513e-02
  4.89319948e-01  6.70510530e-01  3.12500018e-01  7.39215296e-04
  1.97886175e-01  9.99541571e-01  8.47941626e-03  1.03426939e-01
  7.62642844e-02  3.91987734e-01  5.06115604e-01  2.33182614e-01
  2.35621931e-02  6.00000000e+00 -1.00000000e+00  1.00000000e+00]
[5.30933908e-01 7.61808707e-01 8.54448691e-01 5.59139067e-01
 2.85713948e-01 6.37196795e-04 1.78616065e-03 1.70346516e-03
 5.88427157e-01 3.90829331e-02 2.41230301e-01 1.11379682e-02
 2.40121358e-01 6.90421365e-01 3.12500018e-01 6.87290488e-04
 1.43673865e-01 9.99642893e-01 1.44202437e-02 1.94698236e-01
 3.90829331e-02 5.57660955e-01 6.48662019e-01 3.98146356e-01
 8.76325890e-02 9.00000000e+00 6.30000000e+01 1.09000000e+02]
[ 6.76522758e-02  7.37177996e-02  1.42952535e-01  9.32146968e-01
  6.60296896e-01  6.37196795e-04  8.17896357e-04  8

IndexError: index 1170509 is out of bounds for axis 0 with size 1170509

In [25]:
print('The shape of the TRAINING DATA before argumentation is {} and its shape after argumentation is {}'.format(x_train_scaled.shape, x_train_arg.shape))
print('The shape of the TEST DATA before argumentation is {} and its shape after argumentation is {}'.format(x_test_scaled.shape, x_test_arg.shape))
print('The shape of the HIDDEN DATA before argumentation is {} and its shape after argumentation is {}'.format(x_hidden_scaled.shape, x_hidden_arg.shape))

NameError: name 'x_train_arg' is not defined

In [26]:
A = np.load('penalty_matrix.npy')
def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [27]:
#Supervised Algorithms
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsRegressor
from pprint import pprint
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
#Comparing base models accuracies by using k-fold cross validation - 10 folds

from sklearn.model_selection import cross_val_score

model_rf = RandomForestClassifier()
                                  


model_rf.fit(x_train_arg, y_train.values.ravel())
             
train_pred_rf = model_rf.predict(x_train_arg)
open_pred_rf = model_rf.predict(x_test_arg)
hidden_pred_rf = model_rf.predict(x_hidden_arg)
#Printing Reports 


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


NameError: name 'x_train_arg' is not defined

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print('-----------------------TRAIN SET REPORT---------------------')
print("Open set RMSE:", np.sqrt(mean_squared_error(y_train, train_pred_rf)))
print('Open set penalty matrix score:', score(y_train.values, train_pred_rf))
print('Open set report:', classification_report(y_train, train_pred_rf))
print('-----------------------OPEN SET REPORT---------------------')
print("Open set RMSE:", np.sqrt(mean_squared_error(y_test, open_pred_rf)))
print('Open set penalty matrix score:', score(y_test.values, open_pred_rf))
print('Open set report:', classification_report(y_test, open_pred_rf))
print('-----------------------HIDDEN SET REPORT---------------------')
print("Hidden set RMSE:", np.sqrt(mean_squared_error(y_hidden, hidden_pred_rf)))
print('Hidden set penalty matrix score:', score(y_hidden.values, hidden_pred_rf))
print('Hidden set report:', classification_report(y_hidden, hidden_pred_rf))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
facies_colors = ['#F4D03F','#7ccc19','#196F3D','#160599','#2756c4','#3891f0','#80d4ff','#87039e','#ec90fc','#FF4500','#000000','#DC7633']
facies_labels = ['SS', 'S-S', 'SH', 'MR', 'DOL','LIM', 'CH','HAL', 'AN', 'TF', 'CO', 'BS']


#Facies_color_map
facies_color_map = {}
for ind, label in enumerate(facies_labels):
    facies_color_map[label] = facies_colors[ind]
    
def pred_log(logs, well_num, facies_colors, n_pred):
    wells = logs['WELL'].unique()
    logs = logs[logs['WELL'] == wells[well_num]]
    logs = logs.sort_values(by='DEPTH_MD')        #Sorting log by depth
    cmap_facies = colors.ListedColormap(facies_colors[0:len(facies_colors)], 'indexed')
    
    top = logs.DEPTH_MD.min()
    bot = logs.DEPTH_MD.max()
       
    f, ax = plt.subplots(nrows=1, ncols=(12+n_pred), figsize=(15, 12))
    log_colors = ['black', 'red', 'blue', 'green', 'purple','black', 'red', 'blue', 'green', 'purple', 'black', 'red', 'blue', 'green', 'purple', 'black', 'black', 'red', 'blue', 'green', 'purple', 'black', 'black', 'red', 'blue', 'green', 'purple', 'black']

    for i in range(7,18):
      ax[i-7].plot(logs.iloc[:,i], logs.DEPTH_MD, color=log_colors[i])
      ax[i-7].set_ylim(top, bot)
      #ax[i-7].set_xlim(logs.iloc[:,i].min(), logs.iloc[:,i].max())

      ax[i-7].set_xlabel(str(logs.columns[i]))
      ax[i-7].invert_yaxis()
      ax[i-7].grid()

    for j in range((-1-n_pred), 0):
      label = np.repeat(np.expand_dims(logs.iloc[:,j].values, 1), 100, 0)
      im = ax[j].imshow(label, interpolation='none', aspect='auto', cmap=cmap_facies, vmin=0, vmax=12)
      ax[j].set_xlabel(str(logs.columns[j]))

    divider = make_axes_locatable(ax[-1])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((12*' ').join(['SS', 'S-S', 'SH', 'MR', 'DOL','LIM', 'CH','HAL', 'AN', 'TF', 'CO', 'BS']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
        
    f.suptitle('WELL LOGS '+str(wells[well_num]), fontsize=14,y=0.94)

In [ ]:
#Storing results
test_rf_tune = test_imp.copy()
hidden_rf_tune = hidden_imp.copy()
#Saving Results
test_rf_tune['RF_ADD'] = open_pred_rf
hidden_rf_tune['RF_ADD'] = hidden_pred_rf

test_rf_tune.to_csv('test_rf_add.csv')
hidden_rf_tune.to_csv('hidden_rf_add.csv')

In [ ]:
test_rf_tune = pd.read_csv('test_rf_add.csv')
hidden_rf_tune = pd.read_csv('hidden_rf_add.csv')

In [ ]:
x_train_scaled

In [ ]:
#Plotting predictions - HIDDEN DATASET
for i in range(1):
  pred_log(hidden_rf_tune, i, facies_colors, 1)

In [ ]:
from sklearn.metrics import confusion_matrix
    from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
    from itertools import product

    def litho_confusion_matrix(y_true, y_pred):
      facies_dict = {0:'Sandstone', 1:'Sandstone/Shale', 2:'Shale', 3:'Marl',
                    4:'Dolomite', 5:'Limestone', 6:'Chalk', 7:'Halite', 
                    8:'Anhydrite', 9:'Tuff', 10:'Coal', 11:'Basement'}

      # creating a lithofacies names
      labels = list(set(list(y_pred.unique()) + list(y_true.unique())))
      label_names = [facies_dict[k] for k in labels]

      # normalizing confusion matrix by the number of predictions
      cm = pd.DataFrame(confusion_matrix(y_true.values, y_pred.values))
      summ = cm.sum(axis=0)
      cm_norm = pd.DataFrame(np.zeros(cm.shape))
      for i in range(cm.shape[1]):
        for j in range(cm.shape[0]):
          cm_norm[i][j] = cm[i][j]*100/summ[i]
      cm_final = cm_norm.fillna(0).to_numpy()

      fig, ax = plt.subplots(figsize=(12,8))
      plt.imshow(cm_final, interpolation='nearest', cmap=plt.cm.Blues)
      plt.title('NORMALIZED CONFUSION MATRIX', size=15)
      tick_marks = np.arange(len(label_names))
      plt.xticks(tick_marks, label_names, rotation=90)
      plt.yticks(tick_marks, label_names)
      plt.colorbar()
      
      # creating a scores format (black and white)
      fmt = '.2f'
      thresh = cm_final.max() / 2.
      for i, j in product(range(cm_final.shape[0]),   range(cm_final.shape[1])):
        plt.text(j, i, format(cm_final[i, j], fmt),
                      horizontalalignment="center",
                      color="white" if cm_final[i, j] > thresh else "black")
              
      plt.ylabel('True label', size=14)
      plt.xlabel('Predicted label', size=14)

In [ ]:
#Confusion Matrix Function
def my_confusion_matrix(y_true, y_pred):
    labels = np.sort(y_true.unique())
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay

    fig, ax = plt.subplots(figsize=(15,10))
    
    cm = confusion_matrix(y_true, y_pred, sample_weight=None, labels=labels, normalize=None)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels)
    return disp.plot(include_values=True, ax=ax, 
                     xticks_rotation='horizontal', values_format=None)                        #cmap='CMRmap_r'

def default_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(15,10))
    #sns.heatmap(cm, annot=True, linewidths = 0.01)
    sns.heatmap(cm, annot=True)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')